In [20]:
import numpy as np
from scipy.linalg import svd
from ncon import ncon  # import ncon function for tensor contraction

# random seed for reproducibility
np.random.seed(42)

## Syntax:
U, s, Vh = scipy.linalg.svd(A, full_matrices=True, compute_uv=True)

- A: The input matrix (as a NumPy array) to be decomposed.

- full_matrices (optional, boolean): If True (default), U and Vh have shapes (M, M) and (N, N). If False, the shapes are (M, K) and (K, N), where K = min(M, N). Using full_matrices=False is more memory-efficient and is often called "thin SVD".

- compute_uv (optional, boolean): If True (default), the full U and Vh matrices are computed. If False, only the singular values s are computed.

In [21]:
# Create a 3x2 matrix
A = np.array([[1, 2], [3, 4], [5, 6]])

# Decompose the matrix A
U, s, Vh = svd(A)

print("\nU (Left Singular Vectors):")
print(U)

print("\ns (Singular Values as a 1D array):")
print(s)

print("\nVh (Transposed Right Singular Vectors):")
print(Vh)


U (Left Singular Vectors):
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]

s (Singular Values as a 1D array):
[9.52551809 0.51430058]

Vh (Transposed Right Singular Vectors):
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


Note: singular values are returned as 1D array and not matrix.

## Reconstruction of original Matrix
1. Create zero matrix
2. fill diagonal with singular values
3. perform matrix multiplication 

In [22]:
# step 1
Sigma = np.zeros(A.shape)
print("Empty Sigma matrix:", "\n", Sigma)

# step 2, can't use Sigma = np.diag(s), as Sigma is not a square matrix
Sigma[: A.shape[1], : A.shape[1]] = np.diag(s)

# step 3, @ operator is used for matrix multiplication
A_recon = U @ Sigma @ Vh
print("Recostructed A", "\n", A_recon)

assert np.allclose(A, A_recon)
print("\nVerification successful: Reconstructed matrix is close to the original.")

Empty Sigma matrix: 
 [[0. 0.]
 [0. 0.]
 [0. 0.]]
Recostructed A 
 [[1. 2.]
 [3. 4.]
 [5. 6.]]

Verification successful: Reconstructed matrix is close to the original.


## Truncated SVD for Approximation
 key application of SVD is approximating a matrix by using only the top k singular values. This is the basis for techniques like Principal Component Analysis (PCA) and data compression.

In [23]:
# Number of components to keep
k = 1

# Truncate the matrices
U_trunc = U[:, :k]
Sigma_trunc = np.diag(s[:k])
Vh_trunc = Vh[:k, :]

# Reconstruct the approximated matrix
A_approx = U_trunc @ Sigma_trunc @ Vh_trunc

print(f"\nMatrix Approximated with k={k} singular value(s):")
print(A_approx)


Matrix Approximated with k=1 singular value(s):
[[1.35662819 1.71846235]
 [3.09719707 3.92326845]
 [4.83776596 6.12807454]]


# Turning any state into MPOs    

In [24]:
n = 3  # three sites = three legs

# set up random tensor and normalise it
psi = np.random.rand(2**3)
# print("Psi before reshaping: \n ", psi, "\n ")
# psi = psi / np.linalg.norm(psi)  #  normalized state vector, this line is commented as it is useful to test that the code works for unnormalized states, and normalizes them automatically
psi_initial = np.reshape(psi, (2, 2, 2))  # rewrite psi as rank-n tensor
print(
    "Psi normalised and reshaped into 2x2x2 tensor: \n",
    psi,
    "\n Norm:",
    np.linalg.norm(psi),
    "\n",
)

#                      - SITE 1 -
# Step 1 - reshape tensor to matrix, required for svd
psi = np.reshape(psi_initial, (2, 2 ** (n - 1)))
print("Reshape tensor as matrix : \n ", psi)

# Step 2 - SVD to split off first site
U, Lambda, Vd = svd(psi, full_matrices=False)
# print("Shape of U right after SVD: \n", U.shape)

Us = []
# Step 3.5 - add left dummy left virtual index
U = np.reshape(U, (1, 2, 2))  # dummy_mu0, s1, mu1
# print("Shape of U after reshape: \n", U.shape)
Us.append(U)

Psi normalised and reshaped into 2x2x2 tensor: 
 [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615] 
 Norm: 1.6554926857708652 

Reshape tensor as matrix : 
  [[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]]


In [25]:
# Step 3 - Compute the remainder state psi', diagonalise Lambda first since its a 1D array
psi1_remainder = (
    np.diag(Lambda) @ Vd
)  # mu1, (s2 s3)     =>  this has shape (2,4) for a 3 qubits system
print("Psi' remainder", psi1_remainder.shape, psi1_remainder)

#                      - SITE 2 -
# Step 4 - Reshaping, fuse mu1 and s2, this will separate s2 and s3
psi1_remainder = np.reshape(
    psi1_remainder, (4, 2)
)  # (mu1 s2), s3   =>  this has shape (4,2)
print("Psi' remainder reshaped:", psi1_remainder.shape, psi1_remainder)
U, Lambda, Vd = svd(psi1_remainder, full_matrices=False)  # U has shape 4,2

U = np.reshape(U, (2, 2, 2))  # mu1, s2, mu2
Us.append(U)

print("U shape:", U.shape, "Lamda shape:", Lambda.shape, "Vd shape:", Vd.shape)

Psi' remainder (2, 4) [[-0.40467863 -0.91916538 -0.67978374 -0.92448079]
 [-0.02928011 -0.28866414 -0.27763808  0.50397259]]
Psi' remainder reshaped: (4, 2) [[-0.40467863 -0.91916538]
 [-0.67978374 -0.92448079]
 [-0.02928011 -0.28866414]
 [-0.27763808  0.50397259]]
U shape: (2, 2, 2) Lamda shape: (2,) Vd shape: (2, 2)


In [26]:
#                      - SITE 3 -
# Step 5 - Compute remainder psi''
psi2_remainder = np.diag(Lambda) @ Vd  # mu2, s3
print("Psi'' remainder reshaped:", psi2_remainder.shape, psi2_remainder)

Psi'' remainder reshaped: (2, 2) [[-0.69459769 -1.40818727]
 [-0.47047318  0.23206401]]


Because our state vector was already normalized, the singular value in this last SVD is just 1. You could just reshape this matrix into the final tensor (mu2, s3, 1) (hence add additional virtual index) and be done.

However, what if the original state was not normalized? All the "norm" of the state has been pushed into this final matrix. (a good exercise to confirm by skipping the normalization step in the definition of psi above).

In [27]:
def norm_checker(Lamda_tmp, remainder):
    "Controls if the state is normalised by checking if Lamda is 1, if not it does another SVD to handle the norm"
    norm_lambda = np.linalg.norm(Lamda_tmp)
    if np.allclose(norm_lambda, 1, atol=1e-8):
        print("Norm of Lambda is 1, hence the state is normalized.")
        # add right dummy index
        U = np.reshape(remainder, (2, 2, 1))  # Reshape to (mu2, s3, dummy_mu3)

        # check that U is normalized
        if not np.allclose(np.linalg.norm(U), 1):
            print("U is not normalized, something went wrong.")
        Us.append(U)

    else:
        print(
            "Norm of Lambda is not 1, initial state was not normalized. Another SVD was performed to handle the norm."
        )
        # for programmatic elegance and to handle normalization, we do another SVD
        norm_psi2 = np.linalg.norm(remainder)
        print("Norm of psi2_remainder:", norm_psi2)

        # more direct method to Normalize the remainder state
        # remainder = remainder / norm_psi2
        # if np.allclose(np.linalg.norm(remainder), 1):
        #     print("Final state is normalized after normalization step.")
        # else:
        #     print("Final state is not normalized, something went wrong.")

        U, Lambda_final, Vd = svd(remainder, full_matrices=False)
        # now the norm of lambda is the norm of the state
        print("Final Lambda (after SVD):", Lambda_final)
        lambda_final_norm = np.linalg.norm(Lambda_final)

        are_they_equal = np.allclose(lambda_final_norm, norm_psi2)
        print(f"Is the norm of Lambda_final close to norm_psi2?  {are_they_equal}")
        if are_they_equal:
            print("Norm of U:", np.linalg.norm(U))

        # normalise remainder state
        remainder_norm = remainder / lambda_final_norm

        # Add right dummy index
        U = np.reshape(remainder_norm, (2, 2, 1))  # Reshape to (mu2, s3, dummy_mu3)
        Us.append(U)

        print(U.shape, Lambda.shape, Vd.shape)


norm_checker(Lambda, psi2_remainder)

Norm of Lambda is not 1, initial state was not normalized. Another SVD was performed to handle the norm.
Norm of psi2_remainder: 1.6554926857708652
Final Lambda (after SVD): [1.57017748 0.52459385]
Is the norm of Lambda_final close to norm_psi2?  True
Norm of U: 1.414213562373095
(2, 2, 1) (2,) (2, 2)


We can compress the state by only keeping the $\chi$
 largest singular values with their respective singular vectors. This hyper-parameter $\chi$
 is the bond dimension. It allows us to control the amount of entanglement the state can represent between everything that is left and right of the bond 

In [ ]:
# Reconstruction of the original state from the MPS tensors

# shape of Us
for i, U in enumerate(Us):
    print(f"Shape of U at site {i + 1}: {U.shape}")

# 1. Define the list of tensors to contract
tensor_list = [Us[0], Us[1], Us[2]]

# 2. Define the index connections
#    - Positive numbers are contracted. Virtual indices
#    - Negative numbers are the open legs of the final tensor. Physical indices
index_connections = [
    [-1, -2, 1],  # Tensor 0: (dummy_mu0, s1, mu1)
    [1, -3, 2],  # Tensor 1: (mu1, s2, mu2)
    [2, -4, -5],
]  # Tensor 2: (mu2, s3, dummy_mu3)

# 3. Perform the contraction, this is an automated contraction routine
psi_reconstruct_ncon = ncon(tensor_list, index_connections)

print(f"Shape of reconstructed psi from ncon: {psi_reconstruct_ncon.shape}")

# 4. Remove the dummy indices to get the final tensor
# The shape is (1, 2, 2, 2, 1) corresponding to indices (-1, -2, -3, -4, -5)
psi_final = np.reshape(psi_reconstruct_ncon, (2, 2, 2))

print(f"Final shape after removing dummy indices: {psi_final.shape}")

# 5. Verify that the reconstructed tensor matches the original tensor, compute overlap
# first normalise the initial state
psi_initial_norm = psi_initial / np.linalg.norm(psi_initial)

# axes=([0, 1, 2] means that the inner product is taken over all three indices of the tensors
overlap = np.tensordot(psi_initial_norm, psi_final, axes=([0, 1, 2], [0, 1, 2]))
print(f"Overlap between original and reconstructed tensor: {overlap}")

print("\n \n Initial normalised state: \n", psi_initial_norm)
print("\n\n Reconstructed state: \n", psi_final)

Shape of U at site 1: (1, 2, 2)
Shape of U at site 2: (2, 2, 2)
Shape of U at site 3: (2, 2, 1)
Shape of reconstructed psi from ncon: (1, 2, 2, 2, 1)
Final shape after removing dummy indices: (2, 2, 2)
Overlap between original and reconstructed tensor: 1.0

 
 Initial normalised state: 
 [[[0.22624088 0.57427877]
  [0.44216078 0.36161953]]

 [[0.09424303 0.09422846]
  [0.03508539 0.52321351]]]


 Reconstructed state: 
 [[[0.22624088 0.57427877]
  [0.44216078 0.36161953]]

 [[0.09424303 0.09422846]
  [0.03508539 0.52321351]]]


An MPS is in left-canonical form if each of its tensors is a left-normalized isometry.
This means that if you contract a tensor with its own complex conjugate over its physical and left virtual indices, you get an identity matrix.

ADD EQUAtion for the contraction 


In [ ]:
# check if all U satisfies the left-normal isometry condition
for i, U in enumerate(Us):
    # Contract U_tensor with its conjugate over the left and physical indices
    # axes=([0, 1], [0, 1]) means contract axis 0 of the first tensor with axis 0 of the second,
    # and axis 1 of the first with axis 1 of the second.
    # U is real, but for generality we use conjugate
    identity_check = np.tensordot(U, np.conj(U), axes=([0, 1], [0, 1]))
    print(f"Left-canonical check for U at site {i + 1}:")
    # Use the context manager to apply formatting ONLY for this print statement
    with np.printoptions(precision=3, suppress=True):
        print(identity_check)
    print()

Left-canonical check for U at site 1:
[[ 1. -0.]
 [-0.  1.]]

Left-canonical check for U at site 2:
[[1. 0.]
 [0. 1.]]

Left-canonical check for U at site 3:
[[1.]]



TO DO LIST:
- extend to 4 or more physical indixes (4 or more quibits) => make a function that does this trught a loop or something
- Make the tests suggested By Andrew
    - test final inner product
    - check whether uuT* = i
- try right- canonical form
- Try do truncate the singular Values, and see how the reconstructed  tensor looks like
